# Overlaps and Edit Distance - An Analysis



> In computational linguistics and computer science, edit distance is a string metric, i.e. a way of quantifying how dissimilar two strings (e.g., words) are to one another, that is measured by counting the minimum number of operations required to transform one string into the other. Edit distances find applications in natural language processing, where automatic spelling correction can determine candidate corrections for a misspelled word by selecting words from a dictionary that have a low distance to the word in question. In bioinformatics, it can be used to quantify the similarity of DNA sequences, which can be viewed as strings of the letters A, C, G and T.

*[Edit Distance - Wikipedia](https://en.wikipedia.org/wiki/Edit_distance)*



In [ ]:
from Py.geneReader import geneReader

filename = 'SeqFiles/chr1.GRCh38.excerpt.fasta'

data = open ( filename, 'r' )

reads = geneReader ( filename )

data.close ()

In [ ]:
from Py.editDistance import editDistance

import numpy as np

In [ ]:
x = "GATTTACCAGATTGAG"

y = reads

D = [ ]

In [ ]:
# Range covers the offset row plus the length of the pattern

for i in range ( len ( x ) + 1 ) :

    # Initializes the dimensions of the matrix with 0s. 

    D.append ( [ 0 ] * ( len ( y ) + 1 ) )

In [ ]:
print ( 'Length of pattern:', len  ( x ) )

In [ ]:
print ( 'Length of sequence:', len ( y ) )

In [ ]:
D1 = np.matrix ( D )

D1 = D1.view ( )

print ( D1 )

In [ ]:
np.shape ( D )

In [ ]:
for i in range ( len ( x ) + 1 ) :

    D [ i ] [ 0 ] = i

In [ ]:
D1 = np.matrix ( D )

D1 = D1.view ( )

print ( D1 )

In [ ]:
for j in range ( len ( y ) + 1 ) :
        
    D [ 0 ] [ j ] = 0

In [ ]:
D1 = np.matrix ( D )

D1 = D1.view ( )

print ( D1 )

In [ ]:
# Fills in the rest of the matrix rows and columns.
#
# Starts at  row 1. 

for i in range ( 1, len ( x ) + 1 ) :

    # goes by column, starts at column 1

    for j in range ( 1, len ( y ) + 1 ) : 

    # value that is left adjacent to the current value, 
        # plus 1 is the penalty for character skipping

        distHor = D [ i ] [ j - 1 ] + 1 

        # value that is up adjacent to the current value, 
            # plus 1 is the penalty for character skipping

        distVer = D [ i - 1 ] [ j ] + 1

        # edit distance does not further increase if there is a match

            # aka, if matches, does not incur penalty

        if x [ i - 1 ] == y [ j - 1 ] : 

            # Diagonal up/left distance

            distDiag = D [ i - 1 ] [ j - 1 ] 


        # otherwise, diagonal distance value increases by 1

        else :

            distDiag = D [ i - 1 ] [ j - 1 ] + 1 


        # min () takes the minimum edit distance of the 3 possible values
        # so this value will be inserted for the current iteration
        # of row i, column j. 

        D [ i ] [ j ] = min ( distHor, distVer, distDiag ) 

In [ ]:
D1 = np.matrix ( D )

D1 = D1.view ( )

print ( D1 )

In [ ]:
# We are interested in the minimum value of the bottom row.

print ( min ( D [ -1 ] ) )

In [1]:
from Py.geneReader_Q import geneReader_Q

from collections import defaultdict

from Py.overlap import overlap

In [3]:
filename = 'SeqFiles/ERR266411_1.for_asm.fastq'

reads = geneReader_Q ( filename )

?reads

Type:        list
String form: ['TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAA <...> GGCCTCTATTAAGCTCATTCAGGCTTCTGCCGTTTTGGATTTAACCGAAGATGATTTCGATTTTCTGACTAGTAACAAAGTTTGGATTGCTACTG']
Length:      10000
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [4]:
# Starting with an empty set object, we will then add every k-mer association to it

def kmerExtract ( read, k ) :

    setObj = set ()

    for i in range ( 0, len ( read ) - k + 1 ) :

        # We use the add method because we are dealing with a set, 
        # not a list.

        setObj.add ( read [ i : i + k ] )
    
    return setObj

In [5]:
kmerDict = defaultdict ( set )

k = 30

for read in reads : 
    
    for i in range ( 0, len ( read ) - k + 1 ) :
        
        kmerDict [ read [ i : i + k ] ] . add ( read )
        
?kmerDict        

Type:        defaultdict
String form: defaultdict(<class 'set'>, {'TAAACAAGCAGTAGTAATTCCTGCTTTATC': {'GCGCAAGGATAGGTCGAATTTTCTCATTTTCCG <...> ACGCGAACAATTCAGCGGCTTTAACCGGACGGTCGGCCCCGATAATAATGATTGCCGTAAATTCAGGGCTTTCCAGGATTAGGCAGGCCGTTT'}})
Length:      108344
File:        c:\users\ecmos\anaconda3\lib\collections\__init__.py
Docstring:  
defaultdict(default_factory[, ...]) --> dict with default factory

The default factory is called without arguments to produce
a new value when a key is not present, in __getitem__ only.
A defaultdict compares equal to a dict with the same items.
All remaining arguments are treated the same as if they were
passed to the dict constructor, including keyword arguments.

In [6]:
%%time

k = 30

nodeSets = defaultdict ( set )

for a in reads : 
    
    asfx = a [ - k : ]
    
    for b in kmerDict [ a [ - k : ] ] :
        
        if ( b != a and overlap ( a, b, k ) ) :

            nodeSets [ a ] . add ( b )
        
len ( nodeSets )

#refers to reads with at least one overlap with another read
?nodeSets

CPU times: total: 1.72 s
Wall time: 1.73 s


Type:        defaultdict
String form: defaultdict(<class 'set'>, {'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTG <...> TTCTGCCGTTTTGGATTTAACCGAAGATGATTTCGATTTTCTGACTAGTAACAAAGTTTGGATTGCTACTGACCGCTCTCGTGCTCGTCGCTG'}})
Length:      7161
File:        c:\users\ecmos\anaconda3\lib\collections\__init__.py
Docstring:  
defaultdict(default_factory[, ...]) --> dict with default factory

The default factory is called without arguments to produce
a new value when a key is not present, in __getitem__ only.
A defaultdict compares equal to a dict with the same items.
All remaining arguments are treated the same as if they were
passed to the dict constructor, including keyword arguments.

In [9]:
%%time
        
"""

After all reads containing a uniquely specific kmer key are now 
represented in a dictionary of sets, we can then run through 
all of the reads more quickly by just querying the length-k
kmer which contains all values (or entire sequence reads).

If we were checking one by one, every kmer key, the runtime would be extremely long.

For the current read in the list of reads, we pull the length-k 
suffix from the read.

Every key is unique, so that specific k-mer has all the reads we need.
We would then have to cycle one by one, through only that set to append the full length 
sequence reads to the "pairs" list.


"""

k = 30

pairs = [ ]

for a in reads : 
    
    asfx = a [ - k : ]
    
    for b in kmerDict [ asfx ] :
                        
        if ( b != a ) : 
            
            olen = overlap ( a, b, k )

            if ( olen >= k ) :

                pairs.append ( ( a, b ) )
                
?pairs

CPU times: total: 2.38 s
Wall time: 2.43 s


Type:        list
String form: [('TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATA <...> CTTCTGCCGTTTTGGATTTAACCGAAGATGATTTCGATTTTCTGACTAGTAACAAAGTTTGGATTGCTACTGACCGCTCTCGTGCTCGTCGCTG')]
Length:      904746
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.